<a href="https://colab.research.google.com/github/aarjunsrinivasan/CatsvsDogs-Classification/blob/master/Model2_colab_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive and Unzip the Data folder 

## USE GPU IN COLAB

###Uncomment it for extracting

In [0]:
# import zipfile
# with zipfile.ZipFile('/content/Data.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content/drive/My Drive/training')

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import CSVLogger
import tensorflow as tf

In [0]:
logsFile = "logs.csv"
summaryFile = "modelSummary.txt"
modelFile = "trainModel.h5"

'''Insert Data Path Here'''
'''Extract the Data folder as given in the drive link and its path here'''
path = "/content/drive/My Drive/training/Data/"
trainingDirectory = path + "train"
validationDirectory = path + "validation"
#testDirectory = path + "test"

'''Image Data Generator'''
trainingDataGenerator = ImageDataGenerator(rescale=1./255, shear_range=0.1, zoom_range=0.1, horizontal_flip=True)
validationDataGenerator = ImageDataGenerator(rescale=1./255)
#testDataGenerator = ImageDataGenerator(rescale=1./255)

'''Dataset Preparation'''
trainGenerator = trainingDataGenerator.flow_from_directory(trainingDirectory, target_size=(200, 200), batch_size=32, class_mode="binary")
validationGenerator = validationDataGenerator.flow_from_directory(validationDirectory, target_size=(200, 200), batch_size=32, class_mode="binary")

Found 11060 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [0]:
'''CNN Model'''
model = Sequential()

model.add(Conv2D(32, 3, 3, padding='same', input_shape=(200, 200, 3), activation='relu'))
model.add(MaxPooling2D((3, 3)))

model.add(Conv2D(64, 3, 3, padding='same', activation='relu'))
model.add(MaxPooling2D((3, 3)))
model.add(Dropout(0.5))

model.add(Conv2D(64, 3, 3, padding='same', activation='relu'))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))



In [0]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
  with tf.device('/device:GPU:0'):
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
       

In [0]:
hist=model.fit_generator(trainGenerator,
                    steps_per_epoch=len(trainGenerator.filenames) // 32,
                    epochs=1,
                    validation_data=validationGenerator,
                    validation_steps=len(validationGenerator.filenames) // 32,
                    verbose=1)

255/345 [=====================>........] - ETA: 29:21 - loss: 0.3233 - accuracy: 0.9051

In [0]:
import matplotlib.pyplot as plt
f, ax = plt.subplots()
ax.plot([None] + hist.history['loss'], 'o-')
ax.plot([None] + hist.history['val_loss'], 'x-')
ax.legend(['Train Loss', 'Test Loss'], loc = 0)
ax.set_title('Training/Test Loss per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss') 
plt.show()